# TensorFlow Level 101: A Beginner's Complete Guide

Welcome to this comprehensive introduction to TensorFlow!

## What You'll Learn
1. What is TensorFlow?
2. Tensors and operations
3. Building neural networks with Keras
4. Training models
5. Working with real data (tensorflow_sample_data.csv)
6. Model evaluation and visualization

## 1. What is TensorFlow?

**TensorFlow** is Google's open-source ML framework.

| Feature | Benefit |
|---------|--------|
| Keras API | Easy model building |
| Scalable | Laptops to data centers |
| Production-ready | TF Serving, TF Lite |

In [ ]:
# Install packages
!pip install tensorflow numpy pandas matplotlib scikit-learn seaborn

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

## 2. Tensors - The Building Blocks

In [ ]:
# Creating tensors
scalar = tf.constant(42)
vector = tf.constant([1, 2, 3])
matrix = tf.constant([[1, 2], [3, 4]])

print(f"Scalar: {scalar}, shape: {scalar.shape}")
print(f"Vector: {vector}, shape: {vector.shape}")
print(f"Matrix:\n{matrix}")

In [ ]:
# Tensor operations
a = tf.constant([[1., 2.], [3., 4.]])
b = tf.constant([[5., 6.], [7., 8.]])

print(f"Add:\n{tf.add(a, b)}")
print(f"Matmul:\n{tf.matmul(a, b)}")
print(f"Mean: {tf.reduce_mean(a)}")

## 3. Automatic Differentiation

In [ ]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = x ** 2
grad = tape.gradient(y, x)
print(f"y = x^2 at x=3: y={y.numpy()}, dy/dx={grad.numpy()}")

## 4. Building Neural Networks with Keras

In [ ]:
# Sequential API
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(10,)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.summary()

## 5. Working with Real Data: tensorflow_sample_data.csv

Let's build a binary classification model with our 100,000 record dataset!

In [ ]:
# Load dataset
df = pd.read_csv('tensorflow_sample_data.csv')
print(f"Shape: {df.shape}")
print(f"\nLabel distribution:\n{df['label'].value_counts()}")
df.head()

In [ ]:
# Prepare features
num_cols = [f'feature_{i}' for i in range(1, 11)]

# Encode categoricals
le_a, le_b = LabelEncoder(), LabelEncoder()
df['cat_a_enc'] = le_a.fit_transform(df['category_a'])
df['cat_b_enc'] = le_b.fit_transform(df['category_b'])

feature_cols = num_cols + ['cat_a_enc', 'cat_b_enc']
X = df[feature_cols].values
y = df['label'].values

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

In [ ]:
# Build classification model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)
model.summary()

In [ ]:
# Train with early stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Val')
axes[0].set_title('Loss')
axes[0].legend()

axes[1].plot(history.history['accuracy'], label='Train')
axes[1].plot(history.history['val_accuracy'], label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()

axes[2].plot(history.history['auc'], label='Train')
axes[2].plot(history.history['val_auc'], label='Val')
axes[2].set_title('AUC')
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate
results = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")
print(f"Test AUC: {results[2]:.4f}")

In [ ]:
# Predictions and ROC curve
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print("Classification Report:")
print(classification_report(y_test, y_pred))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, lw=2, label=f'AUC = {roc_auc:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## 6. Saving and Loading Models

In [ ]:
# Save model
model.save('classification_model.keras')
print("Model saved!")

# Load model
loaded = keras.models.load_model('classification_model.keras')
print(f"Loaded accuracy: {loaded.evaluate(X_test, y_test, verbose=0)[1]:.4f}")

## Summary

### What We Covered
- TensorFlow basics and tensors
- Building neural networks with Keras
- Training on real data (100,000 records)
- Model evaluation and visualization

### Resources
- [TensorFlow Documentation](https://www.tensorflow.org/)
- [Keras Documentation](https://keras.io/)